In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import math

In [3]:
def simula_horas(mu, sigma, n_simul=7):
    '''Simula los n valores de hoas de sueño'''
    horas_simulados = np.random.normal(mu, sigma, n_simul)
    
    horas_simuladas_filtrado = np.array([i if i < 8 else 8 for i in horas_simulados])
    
    return horas_simuladas_filtrado

In [4]:
def simula_sqi(mu, sigma, n_simul=7, alfa = 0.4):
    '''Simula los n valores de SQI y proporciona el valor de SQI límite para levantar las alertas'''
    sqi_simulados = np.random.normal(mu, sigma, n_simul)
    
    p = norm.ppf((1-alfa)/2)
    valor_alerta = mu+p*sigma
    
    sqi_simulados_filtrado = np.array([i if i < 100 else 100 for i in sqi_simulados])
    
    return sqi_simulados_filtrado, valor_alerta

In [5]:
def model_n(sqi_simulados, valor_alerta, ventana, n_alerta):
    '''Devuelve el ínidice necesario para filtar los SQI simulados hasta que se levanta la alerta. Dependiendo
    de la ventana y el número de veces (n_alerta) que el SQI es inferior al valor de alerta (valor_alerta)'''
    n_simul = len(sqi_simulados)
    alarmas = np.array([sum((sqi_simulados < valor_alerta)[i:i+ventana]) for i in range(n_simul-ventana+1)]) > (n_alerta - 1)
    if np.sum(alarmas) > 0:
        aviso_cambio = np.argmax(alarmas) + ventana
    else:
        aviso_cambio = None
    return aviso_cambio

In [6]:
def nueva_configuracion(presiones, configuracion_cero):
    '''Devuelve la nueva configuración para un individuo dados los estadísticos de las presiones para su categoría'''
    
    # Dataframe con las posibles presiones nuevas
    presiones_posibles = presiones[presiones.frecuencia_relativa > 10].sort_values(by='media', ascending=False)
    for i in range(1, presiones_posibles.shape[0]):
        # Configuración con SQR más alto, para una frecuencia relativa mayor del 10%, distinto de la configuración actual
        nuevas_presiones = presiones_posibles.head(i).index[0]
        if nuevas_presiones != configuracion_cero:
            break
    return nuevas_presiones

In [53]:
def grupo_actual(perfiles_sqr, perfiles_sqr_no_filtrado, sexo, posicion, altura, peso, configuracion_cero):
    '''Se calcula la media y la desviación para la simulación. Además, el dataframe de estadísticos descriptivos
    de las configuraciones de presiones en su grupo'''
    
    # Se calcula el IMC
    IMC = peso / (altura/100)**2
    
    # Cálculo del IMC categorizado
    if IMC < 25:
        IMC_cat = 'Normal'
    elif IMC < 30:
        IMC_cat = 'Overweight'
    else:
        IMC_cat = 'Obese'
        
    # Filtrado del grupo al que pertenece
    perfiles_filtrado = perfiles_sqr[(perfiles_sqr.sexo == sexo) & (perfiles_sqr.posicion == posicion) & (perfiles_sqr.IMC_cat == IMC_cat)]
    horas_perfiles = perfiles_sqr_no_filtrado[(perfiles_sqr_no_filtrado.sexo == sexo) & (perfiles_sqr_no_filtrado.posicion == posicion) & (perfiles_sqr_no_filtrado.IMC_cat == IMC_cat)].copy()
    presiones = perfiles_filtrado[['presiones', 'sqr']].groupby('presiones').describe().loc[:, 'sqr']
    
    # Cálculo de horas de sueño
    horas_perfiles['horas_sueño'] = ((horas_perfiles.fechaFin - horas_perfiles.fechaInicio) / np.timedelta64(1, 's'))/3600
    horas_perfiles['horas_int'] = horas_perfiles['horas_sueño'].astype(int)
    horas_perfiles['horas_int'] = horas_perfiles['horas_int'].apply(lambda x: str(x) if x < 8 else str(8))

    # Cálculo de estadísticos del SQR para cada configuración de presiones
    presiones = presiones.rename({'count':'frecuencia_absoluta', 'mean':'media', 'std': 'desviación'}, axis='columns').round(2)
    presiones['frecuencia_absoluta'] = presiones['frecuencia_absoluta'].astype('int')
    presiones['frecuencia_relativa'] = round(100*presiones['frecuencia_absoluta']/perfiles_filtrado.shape[0], 2)
    try:
        media = presiones.loc[configuracion_cero]['media']
        desviacion = presiones.loc[configuracion_cero]['desviación']
    except:
        media = perfiles_filtrado['sqr'].mean()
        desviacion = perfiles_filtrado['sqr'].std()
        
    if math.isnan(desviacion):
        desviacion = perfiles_filtrado['sqr'].std()
        
    # Cálculo de estadísticos de las horas de sueño
    horas = horas_perfiles[['horas_int', 'sqr']].groupby('horas_int').describe().loc[:, 'sqr']
    horas = horas.rename({'count':'frecuencia_absoluta', 'mean':'media', 'std': 'desviación'}, axis='columns').round(2)
    media_horas = np.mean(horas_perfiles['horas_sueño']).round(2)
    std_horas = np.std(horas_perfiles['horas_sueño']).round(2)
    
    return presiones, round(media, 2), round(desviacion, 2), horas, media_horas, std_horas

In [82]:
def modelo_evolutivo_simulacion(perfiles_sqr, sexo, posicion, altura, peso, configuracion_cero):
    # Se calculan los parámetros de la distribución del SQR y el dataframe de la categoría
    presiones, mu, sigma, horas, muh, sigmah = grupo_actual(perfiles_sqr, perfiles_sqr_no_filtrado, sexo, posicion, altura, peso, configuracion_cero)
    # Se simulan los valores SQR y el valor de alerta
    sqi_simulados, valor_alerta = simula_sqi(mu, sigma)
    # Se simulan los valores de sueño 
    horas_simuladas = simula_horas(muh, sigmah) 
    # Se calcula el primer día de alerta de cada modelo
    alarmas_modelos = [model_n(sqi_simulados, valor_alerta, *i) for i in [(3, 2), (5, 2), (7, 7)] if model_n(sqi_simulados, valor_alerta, *i) != None]
    # Se comprueba si hay alguna alerta en algún modelo 
    if alarmas_modelos != []:
        # Se filtran los valores SQR simulados hasta el primer día de alerta
        valores_prev_alerta = sqi_simulados[:min(alarmas_modelos)]
        valor_medio_horas = np.mean(horas_simuladas[:min(alarmas_modelos)])
        # Se proporciona la configuración recomendada
        nuevas_presiones = nueva_configuracion(presiones, configuracion_cero)
    else:
        valores_prev_alerta = sqi_simulados
        valor_medio_horas = np.mean(horas_simuladas)
        nuevas_presiones = None
    try:
        mejora_horas = round(((horas.loc[str(int(valor_medio_horas)+1)].media  - horas.loc[str(int(valor_medio_horas))].media) /  horas.loc[str(int(valor_medio_horas))].media)*100, 2)
    except:
        mejora_horas = None
    return nuevas_presiones, valores_prev_alerta, valor_alerta, mu, sigma, mejora_horas

In [83]:
# Cargamos datos con SQR
perfiles_sqr = pd.read_parquet('../data/processed/perfiles_sqr_filtrado.parquet')

perfiles_sqr_no_filtrado = pd.read_parquet('../data/processed/perfiles_sqr_filtrado_not_filtered.parquet')

perfiles_sqr_no_filtrado['IMC'] = perfiles_sqr_no_filtrado['peso'] / (perfiles_sqr_no_filtrado['altura']/100)**2
perfiles_sqr_no_filtrado['IMC_cat'] = pd.cut(perfiles_sqr_no_filtrado['IMC'], bins=[0, 25, 30, 50],
                                include_lowest=True,labels=['Normal', 'Overweight', 'Obese'])


perfiles_sqr['IMC'] = perfiles_sqr['peso'] / (perfiles_sqr['altura']/100)**2
perfiles_sqr['IMC_cat'] = pd.cut(perfiles_sqr['IMC'], bins=[0, 25, 30, 50],
                                include_lowest=True,labels=['Normal', 'Overweight', 'Obese'])

In [84]:
sexo = "Female"
posicion = 'Lateral'
altura = 180
peso = 80

## media y sigma del sqi
configuracion_cero = '111111' # no está
# configuracion_cero = '000000' # está sólo una vez
# configuracion_cero = '023211' #está más de una vez

In [85]:
presiones, mu, sigma, horas, muh, sigmah = grupo_actual(perfiles_sqr, perfiles_sqr_no_filtrado, sexo, posicion, altura, peso, configuracion_cero)

In [58]:
horas

,frecuencia_absoluta,media,desviación,min,25%,50%,75%,max
horas_int,,,,,,,,
2,477.0,34.75,13.07,0.16,26.24,36.72,44.65,59.15
3,347.0,40.91,13.82,1.09,34.08,42.93,51.01,65.22
4,381.0,48.87,14.60,1.10,40.80,50.60,59.32,74.51
5,649.0,56.07,15.29,1.55,47.14,58.09,67.80,82.52
6,1409.0,66.44,14.25,0.91,58.30,68.87,77.12,91.18
7,2075.0,73.50,14.46,4.40,64.92,75.59,84.40,99.26
8,3277.0,74.75,14.35,1.01,66.33,75.86,84.62,99.58


In [71]:
horas_simuladas = simula_horas(muh, sigmah)

In [60]:
sqi_simulados, valor_alerta = simula_sqi(mu, sigma)

In [61]:
alarmas_modelos = [model_n(sqi_simulados, valor_alerta, *i) for i in [(3, 2), (5, 2), (7, 7)] if model_n(sqi_simulados, valor_alerta, *i) != None]

In [73]:
if alarmas_modelos != []:
    # Se filtran los valores SQR simulados hasta el primer día de alerta
    valores_prev_alerta = sqi_simulados[:min(alarmas_modelos)]
    valor_medio_horas = np.mean(horas_simuladas[:min(alarmas_modelos)])
    # Se proporciona la configuración recomendada
    nuevas_presiones = nueva_configuracion(presiones, configuracion_cero)
else:
    valores_prev_alerta = sqi_simulados
    valor_medio_horas = np.mean(horas_simuladas)
    nuevas_presiones = None

In [74]:
horas_simuladas

array([8.        , 5.21291719, 7.56524162, 3.14428829, 2.46291141,
       5.25735147, 6.59907504])

In [75]:
valor_medio_horas

6.926052938244333

10.63

In [37]:
sqi_simulados

array([81.068148  , 53.35984711, 61.82951653, 78.97557843, 47.62937881,
       93.09751077, 81.72130671])

In [88]:
modelo_evolutivo_simulacion(perfiles_sqr, sexo, posicion, altura, peso, configuracion_cero)

('012221',
 array([75.47872778, 74.8297257 , 81.63299161, 90.72752336, 53.46359579,
        84.18280791, 50.7197836 ]),
 66.7186326170042,
 74.27,
 14.4,
 10.63)